In [5]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

# Iteration 3. New split method


In [11]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')

vasastan_url = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=925970"
kungsholmen_url = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=925968"
ostermalm_url = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=473448"
sodermalm_url = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=898472"
solna_url = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=18028"
norrmalm_url = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=925969"

url_list = [vasastan_url, kungsholmen_url, ostermalm_url, sodermalm_url, solna_url, norrmalm_url]
area_list = ["vasastan", "kungsholmen", "ostermalm", "sodermalm", "solna", "norrmalm"]

path=r"C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/chromedriver.exe"
driver = webdriver.Chrome(executable_path = path, options=options)

def attribute_extractor(text):
    text = text.lower()
    labels = ["adress", "location", "size:rooms", "fee", "features",  "sale_price", "sold_date", "value_dev", "ppsqm"]
    accomodation_features = []
    
    value_dev_bool = -1
    text_list = text.split("\n")
    
    if text.find("%") == -1: 
        value_dev_bool = 0
    else: 
        value_dev_bool = 1
    
    print(text_list)

    if(len(text_list) < 9):
        if(len(text_list) == 7):
            text_list.insert(4,"null")
            text_list.insert(7, "null")
            accomodation_features = text_list    
        if( (len(text_list) == 8) & (value_dev_bool == 1) ):
            text_list.insert(4,"null")
            accomodation_features = text_list
        elif( (len(text_list) == 8) & (value_dev_bool == 0) ):
            text_list.insert(7, "null")
            accomodation_features = text_list
    elif(len(text_list) == 9):
        if(value_dev_bool == 1):
            accomodation_features = text_list
        elif(value_dev_bool == 0):
            text_list[4] = "balkong&hiss"
            text_list.pop(5)
            accomodation_features = text_list
        
    elif(len(text_list) > 9):
        text_list[4] = "balkong&hiss"
        text_list.pop(5)
        accomodation_features = text_list

    
    housing_info_dict = dict(zip(labels,accomodation_features))
    return housing_info_dict   

def hemnet_scraping(url, num_entries,counter):
    entries_per_page = 50
    outer_list = list(range(0, num_entries, entries_per_page)) 
    big_housing_dict = {}
    
    driver.get(url)
    time.sleep(2)
    if(counter == 0):
        try:
            WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="hcl-button hcl-button--primary"]'))).click()

        except(org.openqa.selenium.StaleElementReferenceException):
            WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="hcl-button hcl-button--primary"]'))).click()
    
    number = driver.find_elements_by_xpath('//*[@class = "result-type-toggle__count--with-upper-bound"]')
    upper_bound = number[0].text
    upper_bound = upper_bound.replace(" ", "")
    upper_bound = int(upper_bound)
    
    if(upper_bound < num_entries):
        num_entries = upper_bound
        outer_list = list(range(0, num_entries, entries_per_page)) 
    
    for adding_number in outer_list:

        elements = driver.find_elements_by_xpath('//*[@class ="sold-property-listing qa-sale-card"]')
        

        for i in range(0,len(elements)):
            big_housing_dict[i+adding_number] = attribute_extractor(elements[i].text)
            variable = elements[i].text
        
        #Identify next page element and scroll down to it
        try:
            next_page = driver.find_element_by_xpath('//*[@class="next_page hcl-button hcl-button--primary hcl-button--full-width"]')
        #driver.execute_script("arguments[0].scrollIntoView();", next_page) 
            time.sleep(1)
        #Change page
            next_page.click()
        except NoSuchElementException:
            print("cant find next button, returning current values")
            return big_housing_dict
    
    return big_housing_dict

counter = 0
for area in url_list:
    housing_info_dict = hemnet_scraping(area, 2500, counter)
    if(counter == 0):
        housing_info_df = pd.DataFrame(housing_info_dict).T
        housing_info_df["district"] = area_list[counter]
        counter = counter + 1
    elif(counter > 0):
        new_housing_info_df = pd.DataFrame(housing_info_dict).T
        new_housing_info_df["district"] = area_list[counter]
        housing_info_df = housing_info_df.append(new_housing_info_df, ignore_index = True)
        counter = counter + 1

housing_info_df.to_csv("stockholm_housing_df.csv",index=False)

C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


['frejgatan 50', 'vasastan, stockholms kommun', '42 m²   2 rum', '2 391 kr/mån', 'balkong', 'slutpris 6 450 000 kr', 'såld 15 januari 2022', '153 571 kr/m²']
['tre liljor 5b, 3,5tr', 'vasastan, stockholms kommun', '37 m²   1,5 rum', '1 998 kr/mån', 'balkong', 'hiss', 'slutpris 4 650 000 kr', 'såld 15 januari 2022', '+16 %', '125 676 kr/m²']
['rådmansgatan 86', 'vasastan, stockholms kommun', '59 m²   2 rum', '2 021 kr/mån', 'uteplats', 'slutpris 7 200 000 kr', 'såld 15 januari 2022', '+13 %', '122 034 kr/m²']
['sankt eriksgatan 109', 'vasastaden, stockholms kommun', '25 m²   1 rum', '436 kr/mån', 'hiss', 'slutpris 3 150 000 kr', 'såld 14 januari 2022', '+19 %', '126 000 kr/m²']
['sveavägen 117b', 'vasastan, stockholms kommun', '36 m²   1,5 rum', '1 273 kr/mån', 'slutpris 4 200 000 kr', 'såld 14 januari 2022', '+17 %', '116 667 kr/m²']
['torsgatan 58, 1,5 tr', 'vasastan - s:t eriksplan, stockholms kommun', '31 m²   1 rum', '1 733 kr/mån', 'hiss', 'slutpris 3 435 000 kr', 'såld 14 januari

['västmannagatan 18, 4 tr', 'vasastan, stockholms kommun', '35 m²   1,5 rum', '2 108 kr/mån', 'balkong', 'hiss', 'slutpris 4 500 000 kr', 'såld 3 januari 2022', '+13 %', '128 571 kr/m²']


C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:96: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


['fridhemsgatan 12', 'kungsholmen, stockholms kommun', '25 m²   1 rum', '1 200 kr/mån', 'uteplats', 'hiss', 'slutpris 3 000 000 kr', 'såld 15 januari 2022', '+11 %', '120 000 kr/m²']
['stagneliusvägen 23, 2 tr', 'fredhäll, stockholms kommun', '47,5 m²   2 rum', '3 278 kr/mån', 'balkong', 'hiss', 'slutpris 4 530 000 kr', 'såld 14 januari 2022', '+13 %', '95 368 kr/m²']
['vitalisvägen 4, vindslägenhet!', 'fredhäll, stockholms kommun', '78 + 16 m²   2 rum', '3 557 kr/mån', 'slutpris 7 650 000 kr', 'såld 14 januari 2022', '+9 %', '98 077 kr/m²']
['sysslomansgatan 24, 2 tr', 'kungsholmen norr mälarstrand, stockholms kommun', '25,5 m²   1 rum', '1 231 kr/mån', 'balkong', 'hiss', 'slutpris 3 100 000 kr', 'såld 14 januari 2022', '121 569 kr/m²']
['alströmergatan 32 m - högst upp!', 'kungsholmen, stockholms kommun', '28 m²   1 rum', '1 534 kr/mån', 'slutpris 3 235 000 kr', 'såld 14 januari 2022', '+30 %', '115 536 kr/m²']
['vitalisvägen 7, 3tr', 'kungsholmen - fredhäll, stockholms kommun', '38 

['garvar lundins gränd 10, 4tr', 'kungsholmen nedre, stockholms kommun', '134 m²   4 rum', '6 994 kr/mån', 'hiss', 'slutpris 15 000 000 kr', 'såld 4 januari 2022', '+6 %', '111 940 kr/m²']
['lindhagensgatan 67', 'thorildsplan, stockholms kommun', '43 m²   2 rum', '2 352 kr/mån', 'slutpris 3 700 000 kr', 'såld 3 januari 2022', '+16 %', '86 047 kr/m²']
['nordenflychtsvägen 62, vån 11', 'kungsholmen, stockholms kommun', '89 m²   4 rum', '2 650 kr/mån', 'balkong', 'hiss', 'slutpris 9 250 000 kr', 'såld 30 december 2021', '+4 %', '103 933 kr/m²']
['fridhemsgatan 55', 'kungsholmen, stockholms kommun', '130 m²   3 rum', '5 940 kr/mån', 'balkong', 'hiss', 'slutpris 13 800 000 kr', 'såld 30 december 2021', '+7 %', '106 154 kr/m²']
['odengatan 6a', 'östermalm / vasastan, stockholms kommun', '38 m²   2 rum', '1 436 kr/mån', 'balkong', 'hiss', 'slutpris 5 600 000 kr', 'såld 15 januari 2022', '+25 %', '147 368 kr/m²']
['valhallavägen 38a, 3tr', 'östermalm, stockholms kommun', '39 m²   1,5 rum', '2 

['sibyllegatan 5', 'östermalm, stockholms kommun', '27 m²   1 rum', '895 kr/mån', 'slutpris 3 900 000 kr', 'såld 7 december 2021', '+30 %', '144 444 kr/m²']
['valhallavägen 152 c, 3 tr', 'östermalm, stockholms kommun', '79 m²   3 rum', '3 085 kr/mån', 'balkong', 'hiss', 'slutpris 9 200 000 kr', 'såld 7 december 2021', '+11 %', '116 456 kr/m²']
['östermalmsgatan 75a', 'östermalm, stockholms kommun', '50 m²   2 rum', '2 694 kr/mån', 'balkong', 'slutpris 7 000 000 kr', 'såld 7 december 2021', '+12 %', '140 000 kr/m²']
['nybergsgatan 3', 'östermalm, stockholms kommun', '73 m²   3 rum', '0 kr/mån', 'balkong', 'slutpris 11 950 000 kr', 'såld 7 december 2021', '+9 %', '163 699 kr/m²']
['artillerigatan 63', 'östermalm, stockholms kommun', '33 m²   1 rum', '1 579 kr/mån', 'hiss', 'slutpris 4 225 000 kr', 'såld 7 december 2021', '+13 %', '128 030 kr/m²']
['valhallavägen 135', 'östermalm/nedre gärdet, stockholms kommun', '57,5 m²   3 rum', '2 816 kr/mån', 'balkong', 'hiss', 'slutpris 6 300 000 kr

['åsögatan 194', 'södermalm - sofia, stockholms kommun', '49 m²   2 rum', '2 394 kr/mån', 'hiss', 'slutpris 4 900 000 kr', 'såld 6 januari 2022', '+18 %', '100 000 kr/m²']
['tantogatan 27- accepterat pris.', 'södermalm, stockholms kommun', '83 m²   3 rum', '3 595 kr/mån', 'balkong', 'hiss', 'slutpris 6 600 000 kr', 'såld 5 januari 2022', '±0 %', '79 518 kr/m²']
['hornsgatan 85, 5tr', 'maria/gamla stan/högalid, stockholms kommun', '99 m²   3 rum', '4 295 kr/mån', 'hiss', 'slutpris 11 200 000 kr', 'såld 5 januari 2022', '+28 %', '113 131 kr/m²']
['åsögatan 120', 'södermalm katarina, stockholms kommun', '28 m²   2 rum', '846 kr/mån', 'hiss', 'slutpris 3 500 000 kr', 'såld 4 januari 2022', '+27 %', '125 000 kr/m²']
['kapellgränd 6a, nb, ög', 'södermalm mosebacke, stockholms kommun', '37 m²   1 rum', '1 300 kr/mån', 'hiss', 'slutpris 4 250 000 kr', 'såld 4 januari 2022', '114 865 kr/m²']
['heleneborgsgatan 8, 0,5 tr', 'högalid, stockholms kommun', '37 m²   1 rum', '2 244 kr/mån', 'slutpris 

['bangatan 5a', 'huvudsta, solna kommun', '66 m²   3 rum', '3 505 kr/mån', 'balkong', 'slutpris 3 630 000 kr', 'såld 11 januari 2022', '+30 %', '55 000 kr/m²']
['ankdammsgatan 31', 'skytteholm, solna kommun', '61 m²   2 rum', '3 702 kr/mån', 'balkong', 'hiss', 'slutpris 3 150 000 kr', 'såld 11 januari 2022', '+13 %', '51 639 kr/m²']
['honnörsgatan 1,bv', 'järvastaden, solna kommun', '53 m²   2 rum', '3 261 kr/mån', 'uteplats', 'hiss', 'slutpris 2 800 000 kr', 'såld 11 januari 2022', '+12 %', '52 830 kr/m²']
['tegnérgatan 57b, 3tr', 'vasastan - city/norrmalm, stockholms kommun', '26 m²   1 rum', '1 462 kr/mån', 'hiss', 'slutpris 3 060 000 kr', 'såld 14 januari 2022', '-1 %', '117 692 kr/m²']
['tegnérgatan 55a', 'vasastan - city/norrmalm, stockholms kommun', '35 m²   1,5 rum', '2 257 kr/mån', 'slutpris 3 950 000 kr', 'såld 13 januari 2022', '+13 %', '112 857 kr/m²']
['kammakargatan 70, 3tr', 'vasastan - city/norrmalm, stockholms kommun', '29 m²   1 rum', '1 208 kr/mån', 'balkong', 'hiss'

['upplandsgatan 11a', 'vasastan, stockholms kommun', '55 m²   2 rum', '2 393 kr/mån', 'balkong', 'slutpris 7 175 000 kr', 'såld 7 november 2021', '+22 %', '130 455 kr/m²']
['drottninggatan 73 c, 5tr vindsvåning', 'vasastan - city/norrmalm, stockholms kommun', '112 + 24 m²   3 rum', '787 kr/mån', 'hiss', 'slutpris 12 700 000 kr', 'såld 3 november 2021', '±0 %', '113 393 kr/m²']
['herkulesgatan 26, vån 11 / 11', 'norrmalm, stockholms kommun', '70 m²   3 rum', '3 864 kr/mån', 'balkong', 'hiss', 'slutpris 7 000 000 kr', 'såld 3 november 2021', '+9 %', '100 000 kr/m²']
['regeringsgatan 80', 'city - norrmalm - vasastan, stockholms kommun', '43 m²   1,5 rum', '0 kr/mån', 'hiss', 'slutpris 5 530 000 kr', 'såld 3 november 2021', '+29 %', '128 605 kr/m²']
['torsgatan 7', 'vasastan, stockholms kommun', '38,5 m²   1,5 rum', '2 047 kr/mån', 'balkong', 'hiss', 'slutpris 4 000 000 kr', 'såld 3 november 2021', '+25 %', '103 896 kr/m²']
['regeringsgatan 70a, 3tr', 'city norrmalm vasastan, stockholms ko

In [12]:
housing_info_df

,adress,location,size:rooms,fee,features,sale_price,sold_date,value_dev,ppsqm,district
0,frejgatan 50,"vasastan, stockholms kommun",42 m² 2 rum,2 391 kr/mån,balkong,slutpris 6 450 000 kr,såld 15 januari 2022,null,153 571 kr/m²,vasastan
1,"tre liljor 5b, 3,5tr","vasastan, stockholms kommun","37 m² 1,5 rum",1 998 kr/mån,balkong&hiss,slutpris 4 650 000 kr,såld 15 januari 2022,+16 %,125 676 kr/m²,vasastan
2,rådmansgatan 86,"vasastan, stockholms kommun",59 m² 2 rum,2 021 kr/mån,uteplats,slutpris 7 200 000 kr,såld 15 januari 2022,+13 %,122 034 kr/m²,vasastan
3,sankt eriksgatan 109,"vasastaden, stockholms kommun",25 m² 1 rum,436 kr/mån,hiss,slutpris 3 150 000 kr,såld 14 januari 2022,+19 %,126 000 kr/m²,vasastan
4,sveavägen 117b,"vasastan, stockholms kommun","36 m² 1,5 rum",1 273 kr/mån,null,slutpris 4 200 000 kr,såld 14 januari 2022,+17 %,116 667 kr/m²,vasastan
...,...,...,...,...,...,...,...,...,...,...
295,"drottninggatan 73 c, 5tr vindsvåning","vasastan - city/norrmalm, stockholms kommun",112 + 24 m² 3 rum,787 kr/mån,hiss,slutpris 12 700 000 kr,såld 3 november 2021,±0 %,113 393 kr/m²,norrmalm
296,"herkulesgatan 26, vån 11 / 11","norrmalm, stockholms kommun",70 m² 3 rum,3 864 kr/mån,balkong&hiss,slutpris 7 000 000 kr,såld 3 november 2021,+9 %,100 000 kr/m²,norrmalm
297,regeringsgatan 80,"city - norrmalm - vasastan, stockholms kommun","43 m² 1,5 rum",0 kr/mån,hiss,slutpris 5 530 000 kr,såld 3 november 2021,+29 %,128 605 kr/m²,norrmalm
298,torsgatan 7,"vasastan, stockholms kommun","38,5 m² 1,5 rum",2 047 kr/mån,balkong&hiss,slutpris 4 000 000 kr,såld 3 november 2021,+25 %,103 896 kr/m²,norrmalm


# Old iteration. Scraping Vasastan succesfully (although each page is being scrubbed, and that is taking a long long time to process)

## This solution is very rendudant to acquire some more features, although, it takes too long to run atm...


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

vasastan_url = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=925970"

path=r"C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/chromedriver.exe"
driver = webdriver.Chrome(executable_path = path, options=options)

def hemnet_scraping(url, num_entries):
    try:
        driver.get(vasastan_url)
        time.sleep(2)
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="hcl-button hcl-button--primary"]'))).click()

        except(org.openqa.selenium.StaleElementReferenceException):
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="hcl-button hcl-button--primary"]'))).click()

        time.sleep(2)
        elements = driver.find_elements_by_class_name("sold-property-link")
        housing_dict = {}
        #50 entries per page, so at the count of 50, the page has to change
        entries_per_page = 50
        outer_list = list(range(0, num_entries, entries_per_page)) 

        for outer in outer_list:
            elements = driver.find_elements_by_class_name("sold-property-link")
            for i in range(0,entries_per_page-1):
                print("DETTA I ÄR DET JUST NU: ", i , "\n", "\n")
                print("HUr lång är elements? : ", len(elements))
                housing_info_text = get_features_from_hemnet(i, elements)
                housing_dict[i+outer] = attribute_extractor(housing_info_text)
                time.sleep(2)
                driver.close();
                time.sleep(2)
                driver.switch_to.window(driver.window_handles[0]);
                time.sleep(2)
            #Identify next page element and scroll down to it
            next_page = driver.find_element_by_xpath('//*[@class="next_page hcl-button hcl-button--primary hcl-button--full-width"]')
            #driver.execute_script("arguments[0].scrollIntoView();", next_page) 
            time.sleep(1)
            #Change page
            next_page.click()
    except Exception as e:
        print(e, "caught at: ", outer, "" , i) 
        return housing_dict
    
    return housing_dict


def get_features_from_hemnet(index, elements):
    time.sleep(1)
    elements[index].click()
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)

    adress = driver.find_element_by_xpath("//html/body/div[3]/div[2]/div/div[1]/h1").get_attribute('textContent')
    adress = adress.replace("\n  Slutpris\n  ", "")

    price = driver.find_element_by_class_name("sold-property__price-value")
    price = price.text

    time_sold = driver.find_element_by_xpath("//html/body/div[3]/div[2]/div/div[1]/p/time").get_attribute("textContent")
    
    print(time_sold)
    attributes = driver.find_element_by_class_name("sold-property__attributes")
    attributes_text = attributes.text

    stats = driver.find_element_by_class_name("sold-property__price-stats")
    stats_text = stats.text

    housing_info_text = "Pris\n"+price+ "\nadress\n"+ adress + "Datum såld:" + time_sold + attributes_text + "\n" + stats_text
    return housing_info_text 

def attribute_extractor(text):
    
    #text = text_element_list.text
    word_list = []
    while text.find("\n") != -1:
        cut_off = text.find("\n")
        word_list.append(text[0:cut_off])
        text = text[cut_off+1:len(text)]   
    word_list.append(text)

    attribute_list = []
    attribute_values = []


    for i in range(0, len(word_list)):
        if i % 2 == 0:
            attribute_list.append(word_list[i])
        else:
            attribute_values.append(word_list[i])

    res = dict(zip(attribute_list, attribute_values))
    return res

#How many entries we want to extract
num_entries = 2000
vasastan_dict = hemnet_scraping(vasastan_url, num_entries)

vasastan_df = pd.DataFrame(vasastan_dict)
vasastan_df = vasastan_df.T

vasastan_df.to_csv(index=False)

In [1]:
!pipreqs --force "C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing"

INFO: Successfully saved requirements file in C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing\requirements.txt
